# Automatic algorithm/strategy selection
Here we will show you how you can perform an algorithm selection, helping you choose the best optimization algorithm for your specific problem.

In [2]:
from metaheuristic_designer import simple
from metaheuristic_designer.operators import OperatorVector
from metaheuristic_designer.selectionMethods import ParentSelection, SurvivorSelection
from metaheuristic_designer.algorithms import AlgorithmSelection, StrategySelection
from metaheuristic_designer.initializers import UniformVectorInitializer
from metaheuristic_designer.strategies import HillClimb, SA, ES, GA, DE, PSO, RandomSearch
from metaheuristic_designer.benchmarks import *
import numpy as np

For the following examples, we will use the Rastrigin function for which we need to find the minimum value. This is a very hard optimization problem and will allow us to compare the performance of different algorithms.

In [3]:
rastrigin_func = Rastrigin(size=2)

## Algorithm selection
We will first explore the AlgorithmSelection class, which launches each of the specified algorithms multiple times and gets some statistics about the evaluation.

We begin by defining the algorithm we are going to use. The ```simple``` package gives us premade implementations of well known optimization algorithms with a single function call, although customization is limited.

In [5]:
# The parameters common to all algorithms
params = {
    # Stopping condition
    "stop_cond": "neval",
    "neval": 5e4,
    # Encoding
    "encoding": "real",
    # Disable verbose, the output gets very cluttered otherwise
    "verbose": False,
    # Number of times to launch each algorithm
    "repetitions": 10,
}

# Define the algorithms we are going to use
algorithms = [
    simple.random_search(params, rastrigin_func),
    simple.hill_climb(params, rastrigin_func),
    simple.simulated_annealing(params, rastrigin_func),
    simple.evolution_strategy(params, rastrigin_func),
    simple.genetic_algorithm(params, rastrigin_func),
    simple.differential_evolution(params, rastrigin_func),
    simple.particle_swarm(params, rastrigin_func),
]

# Instanciate the AlgorithmSelection class
algorithm_search = AlgorithmSelection(algorithms)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

Running 7 algorithms 10 times each.


The optimize function should have returned the best overall solution and a report with statistics about the execution

In [6]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

best solution: [-3.19603294e-09 -1.59301793e-09]
fitness: 0.0


In [7]:
report.sort_values("fitness_avg")

,name,realtime_min,realtime_avg,realtime_max,realtime_std,cputime_min,cputime_avg,cputime_max,cputime_std,fitness_min,fitness_avg,fitness_max,fitness_std
5,DE,0.562943,0.578511,0.618806,0.014999,0.556888,0.576334,0.617165,0.015587,0.000000,0.000000,0.000000,0.000000
6,PSO,0.519878,0.533419,0.540672,0.007329,0.515027,0.531193,0.539417,0.008308,0.000000,0.000000,0.000000,0.000000
0,RandomSearch,1.272375,1.287468,1.315041,0.013076,1.268989,1.285139,1.312198,0.013134,0.017069,0.113032,0.254730,0.086504
4,GA,1.678561,1.751952,2.175720,0.152063,1.671410,1.746212,2.169949,0.152076,0.134047,1.187095,2.742185,0.926129
3,ES,0.694241,0.706379,0.715866,0.006808,0.692930,0.703567,0.713029,0.007174,1.214225,4.101807,8.328063,2.686718
2,SA,29.571356,31.733815,34.983758,1.866453,29.485885,31.642117,34.912028,1.876310,5.003792,24.219868,52.783766,12.951277
1,HillClimb,31.804156,32.854692,34.071966,0.818317,31.702420,32.761961,33.975877,0.817295,2.832885,24.408948,58.709020,17.176153


With this, we can clearly see that the best algorithm for this task seems to be the Differential Evolution algorithm, and surprisingly, RandomSearch performs better than HillClimb, SA, ES and even GA. This could be caused by some misconfiguration of parameters, but since we just used the default implementation, we have no way of changing this.

## Strategy Selection
We could have specified the algorithms directly constructing them piece by piece, but unless you need to try some specific implementation of an algorithm, it is much simpler to search for the best Search Strategy. This is where the StrategySearch class is used. It is very similar to the AlgorithmSelection class, but using search strategies.

This also has the advantage that you can choose a custom name for each strategy.

In [8]:
# Initializer for sinlge-solution algorithms
single_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=1)

# Initializer for population-based algorithms
pop_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=100)

# Define strategies to be tested
strategies = [
    RandomSearch(pop_initializer),
    HillClimb(
        single_initializer,
        perturb_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        name="HillClimb",
    ),
    SA(
        single_initializer,
        perturb_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="SA",
    ),
    SA(
        pop_initializer,
        perturb_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="ParallelSA",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 5e-3}),
        selection_op=SurvivorSelection("(m+n)"),
        params={"offspringSize": 150},
        name="ES-(μ+λ)",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 5e-2}),
        selection_op=SurvivorSelection("(m,n)"),
        params={"offspringSize": 700},
        name="ES-(μ,λ)",
    ),
    GA(
        pop_initializer,
        mutation_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        cross_op=OperatorReal("Multipoint"),
        parent_sel_op=ParentSelection("Tournament", {"amount": 60, "p": 0.1}),
        selection_op=SurvivorSelection("Elitism", {"amount": 10}),
        params={"pcross": 0.9, "pmut": 0.1},
        name="GA",
    ),
    DE(
        pop_initializer,
        OperatorReal("DE/best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/best/1",
    ),
    DE(
        pop_initializer,
        OperatorReal("DE/rand/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/rand/1",
    ),
    DE(
        pop_initializer,
        OperatorReal("DE/current-to-best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/current-to-best/1",
    ),
    PSO(pop_initializer, {"w": 0.7, "c1": 1.5, "c2": 1.5}, name="PSO"),
]

# Instanciate the StrategySelection class
algorithm_search = StrategySelection(
    rastrigin_func,
    strategies,
    algorithm_params={
        "stop_cond": "neval",
        "neval": 5e4,
        "verbose": False,
    },
    params={"verbose": True, "repetitions": 10},
)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

NameError: name 'OperatorReal' is not defined

In [ ]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

best solution: [-1.26199997e-09 -4.53316307e-10]
fitness: 0.0


In [ ]:
report.sort_values("fitness_avg")

,name,realtime_min,realtime_avg,realtime_max,realtime_std,cputime_min,cputime_avg,cputime_max,cputime_std,fitness_min,fitness_avg,fitness_max,fitness_std
7,DE/best/1,1.519729,1.548633,1.608609,0.025094,1.519781,1.565137,1.628799,0.035645,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
8,DE/rand/1,1.123156,1.167058,1.297242,0.058852,1.128656,1.174887,1.296135,0.055647,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
9,DE/current-to-best/1,1.584857,1.613552,1.653009,0.020102,1.597510,1.635562,1.727576,0.037934,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
10,PSO,1.115229,1.156385,1.321878,0.061156,1.115460,1.169345,1.320756,0.061212,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
6,GA,1.769814,1.803421,1.826834,0.016155,1.768550,1.802076,1.825370,0.016126,1.526814e-10,1.526814e-10,1.526814e-10,0.000000
5,"ES-(μ,λ)",0.930633,0.943650,0.991546,0.017881,0.936875,0.948075,0.992188,0.016175,1.081313e-06,1.081313e-06,1.081313e-06,0.000000
0,RandomSearch,1.252175,1.293047,1.346313,0.033340,1.253476,1.291402,1.343438,0.032443,3.427405e-03,8.270857e-03,1.034662e-02,0.003342
4,ES-(μ+λ),0.922964,0.929995,0.944922,0.005929,0.924595,0.945595,1.008896,0.027014,8.391705e-07,3.979843e-01,1.989918e+00,0.839023
3,ParallelSA,0.906290,0.918373,0.938245,0.010232,0.907828,0.938450,1.008967,0.034142,2.701125e-04,5.977198e-01,9.949638e-01,0.426150
1,HillClimb,1.124578,1.142825,1.157456,0.010962,1.124174,1.143407,1.158496,0.011480,9.949591e-01,4.676294e+00,1.691420e+01,4.475443
